In [ ]:
!pip install -q ultralytics

In [ ]:
model_path = '/content/best_sku110K_object_identification_yolov11n_model.pt'
from ultralytics import YOLO
model = YOLO(model_path)

In [ ]:
image_path = '/content/test_2847.jpg'

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
results = model(image_path)
# Adjust font_size and line_width to make annotations smaller
annotated_image = results[0].plot(font_size=8, line_width=1)
cv2_imshow(annotated_image)

In [ ]:
#@title Sum of area of bounding boxes
import cv2

print("\n--- Bounding Box Areas ---")
total_boxes_area = 0

# Load the original image to get its dimensions
original_image = cv2.imread(image_path)
image_height, image_width, _ = original_image.shape
total_image_area = image_height * image_width

for r in results:
    boxes = r.boxes.xyxy.cpu().numpy()
    confidences = r.boxes.conf.cpu().numpy()
    class_ids = r.boxes.cls.cpu().numpy()
    names = r.names

    for i in range(len(boxes)):
        x1, y1, x2, y2 = boxes[i]
        # Calculate area: (width) * (height)
        area = (x2 - x1) * (y2 - y1)
        class_name = names[int(class_ids[i])]
        confidence = confidences[i]

        # print(f"Object: {class_name}, Confidence: {confidence:.2f}, Area: {area:.2f} pixels")
        total_boxes_area += area

print(f"\n--- Summary ---")
print(f"Total area covered by all bounding boxes: {total_boxes_area:.2f} pixels")
print(f"Total image area: {total_image_area:.2f} pixels")

percentage_of_image_covered = (total_boxes_area / total_image_area) * 100
print(f"Percentage of image covered by bounding boxes: {percentage_of_image_covered:.2f}%")

In [ ]:
#@title
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

# Load the original image
original_image = cv2.imread(image_path)
# Make a copy to draw on, as cv2.rectangle modifies the image in-place
annotated_custom_image = original_image.copy()

# Define a color for the bounding boxes (B, G, R) - e.g., Green
bbox_color = (0, 255, 0) # Green color
text_color = (255, 255, 255) # White text

# Loop through each detection result to draw custom bounding boxes
for r in results:
    boxes = r.boxes.xyxy.cpu().numpy()
    confidences = r.boxes.conf.cpu().numpy()
    class_ids = r.boxes.cls.cpu().numpy()
    names = r.names

    for i in range(len(boxes)):
        x1, y1, x2, y2 = map(int, boxes[i]) # Convert coordinates to integers
        confidence = confidences[i]
        class_id = int(class_ids[i])
        class_name = names[class_id]
        area = (x2 - x1) * (y2 - y1) # Calculate area

        # Draw rectangle
        cv2.rectangle(annotated_custom_image, (x1, y1), (x2, y2), bbox_color, 2)

        # Prepare label text to include area
        label = f"{class_name}: {confidence:.2f}, Area: {area:.0f} px"

        # Get text size for background rectangle
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.8
        font_thickness = 2
        (text_width, text_height), baseline = cv2.getTextSize(label, font, font_scale, font_thickness)

        # Draw background rectangle for text
        cv2.rectangle(annotated_custom_image, (x1, y1 - text_height - baseline), (x1 + text_width, y1), bbox_color, -1)
        # Put text (label, confidence, and area)
        cv2.putText(annotated_custom_image, label, (x1, y1 - baseline), font, font_scale, text_color, font_thickness, cv2.LINE_AA)

# Display the image with custom annotations
cv2_imshow(annotated_custom_image)

In [ ]:
#@title Functions for rows and gaps

import numpy as np

def cluster_rows(bboxes, row_tol=50):
    """
    bboxes: list of [x1, y1, x2, y2]
    row_tol: vertical tolerance for bottom edges to group into shelves
    """
    bottoms = np.array([y2 for (x1, y1, x2, y2) in bboxes])
    order = np.argsort(bottoms)  # sort by bottom edge

    rows = []
    current = [order[0]]

    for idx in order[1:]:
        prev = current[-1]
        if abs(bottoms[idx] - bottoms[prev]) <= row_tol:
            current.append(idx)
        else:
            rows.append(current)
            current = [idx]

    rows.append(current)
    return rows


def find_gaps_in_row(row_bboxes, width_factor=0.5):
    """
    row_bboxes: list of [x1,y1,x2,y2] for a single shelf row
    width_factor: threshold relative to median width
    """
    # sort left-to-right
    row_bboxes = sorted(row_bboxes, key=lambda b: b[0])
    widths = [b[2] - b[0] for b in row_bboxes]
    median_w = np.median(widths)

    gaps = []
    for i in range(len(row_bboxes)-1):
        cur = row_bboxes[i]
        nxt = row_bboxes[i+1]
        gap = nxt[0] - cur[2]

        if gap > median_w * width_factor:
            # gap detected
            gap_box = [
                cur[2],
                min(cur[1], nxt[1]),
                nxt[0],
                max(cur[3], nxt[3])
            ]
            gaps.append(gap_box)

    return gaps


def calculate_iou(box1, box2):
    """
    Calculates the Intersection over Union (IoU) of two bounding boxes.

    Args:
        box1 (list or np.array): First bounding box in [x1, y1, x2, y2] format.
        box2 (list or np.array): Second bounding box in [x1, y1, x2, y2] format.

    Returns:
        float: The IoU value, or 0.0 if there is no overlap or union area is zero.
    """

    # Determine the coordinates of the intersection rectangle
    x1_box1, y1_box1, x2_box1, y2_box1 = box1
    x1_box2, y1_box2, x2_box2, y2_box2 = box2

    x_left = max(x1_box1, x1_box2)
    y_top = max(y1_box1, y1_box2)
    x_right = min(x2_box1, x2_box2)
    y_bottom = min(y2_box1, y2_box2)

    # Calculate the area of the intersection rectangle
    if x_right < x_left or y_bottom < y_top:
        area_intersection = 0.0
    else:
        area_intersection = (x_right - x_left) * (y_bottom - y_top)

    # Calculate the area of each bounding box
    area_box1 = (x2_box1 - x1_box1) * (y2_box1 - y1_box1)
    area_box2 = (x2_box2 - x1_box2) * (y2_box2 - y1_box2)

    # Calculate the Union area
    area_union = float(area_box1 + area_box2 - area_intersection)

    # Handle division by zero
    if area_union == 0:
        return 0.0

    # Return the IoU
    iou = area_intersection / area_union
    return iou

def categorize_gaps(empty_shelf_gaps, all_bboxes_list, iou_threshold):
    """
    Categorizes empty shelf gaps into overlapping and non-overlapping groups
    based on Intersection over Union (IoU) with detected object bounding boxes.

    Args:
        empty_shelf_gaps (list): A list of bounding boxes representing empty shelf gaps.
        all_bboxes_list (list): A list of bounding boxes representing detected objects.
        iou_threshold (float): The IoU threshold to determine if a gap overlaps with an object.

    Returns:
        tuple: A tuple containing two lists:
               - overlapping_gaps (list): Gaps that overlap with any object above the threshold.
               - non_overlapping_gaps (list): Gaps that do not overlap with any object above the threshold.
    """
    overlapping_gaps = []
    non_overlapping_gaps = []

    for gap_box in empty_shelf_gaps:
        is_overlapping = False
        for object_box in all_bboxes_list:
            iou = calculate_iou(gap_box, object_box)
            if iou > iou_threshold:
                is_overlapping = True
                break # Found an overlap for this gap, no need to check other objects

        if is_overlapping:
            overlapping_gaps.append(gap_box)
        else:
            non_overlapping_gaps.append(gap_box)

    return overlapping_gaps, non_overlapping_gaps

print("The `categorize_gaps` function has been defined.")



In [ ]:
all_bboxes = []
for r in results:
    # Assuming 'results' contains a list of detection objects, each with a .boxes attribute
    # .xyxy.cpu().numpy() extracts bounding boxes in [x1, y1, x2, y2] format
    all_bboxes.extend(r.boxes.xyxy.cpu().numpy())

# Convert to a list of lists/tuples for easier handling if numpy arrays cause issues with list appends later
all_bboxes_list = [list(bbox) for bbox in all_bboxes]

# Cluster bounding boxes into rows
rows = cluster_rows(all_bboxes_list, row_tol=50)

print("Detected Gaps:")
empty_shelf_gaps = []
for i, row_indices in enumerate(rows):
    current_row_bboxes = [all_bboxes_list[j] for j in row_indices]
    gaps = find_gaps_in_row(current_row_bboxes, width_factor=0.5)
    if gaps:
        print(f"  Row {i+1}:")
        for gap in gaps:
            print(f"    Gap BBox: [x1={gap[0]:.0f}, y1={gap[1]:.0f}, x2={gap[2]:.0f}, y2={gap[3]:.0f}]")
            empty_shelf_gaps.append(gap)

if not empty_shelf_gaps:
    print("No significant gaps detected based on current parameters.")

iou_threshold = 0.1 # Updated threshold as per user request
overlapping_gaps, non_overlapping_gaps = categorize_gaps(empty_shelf_gaps, all_bboxes_list, iou_threshold)

print(f"Total empty shelf gaps: {len(empty_shelf_gaps)}")
print(f"Gaps overlapping with objects (IoU > {iou_threshold}): {len(overlapping_gaps)}")
print(f"Gaps not overlapping with objects: {len(non_overlapping_gaps)}")
# You can now use 'empty_shelf_gaps' for further visualization or analysis

In [ ]:
# @title
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

# Load the original image
original_image_for_gaps = cv2.imread(image_path)
# Make a copy to draw on
annotated_image_with_gaps = original_image_for_gaps.copy()

# Define a color for the gap bounding boxes (B, G, R) - e.g., Red
gap_bbox_color = (0, 0, 255) # Red color
gap_text_color = (255, 255, 255) # White text for labels

# Draw each gap bounding box
if empty_shelf_gaps:
    for gap in empty_shelf_gaps:
        x1, y1, x2, y2 = map(int, gap) # Convert coordinates to integers

        # Draw rectangle for the gap
        cv2.rectangle(annotated_image_with_gaps, (x1, y1), (x2, y2), gap_bbox_color, 2)

        # Optional: Add a label for the gap
        label = "EMPTY GAP"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.6
        font_thickness = 1
        (text_width, text_height), baseline = cv2.getTextSize(label, font, font_scale, font_thickness)

        # Draw background rectangle for text
        cv2.rectangle(annotated_image_with_gaps, (x1, y1 - text_height - baseline), (x1 + text_width, y1), gap_bbox_color, -1)
        # Put text
        cv2.putText(annotated_image_with_gaps, label, (x1, y1 - baseline), font, font_scale, gap_text_color, font_thickness, cv2.LINE_AA)
else:
    print("No empty shelf gaps to draw.")

# Display the image with gap annotations
cv2_imshow(annotated_image_with_gaps)


In [ ]:
#@title Need to remove boxes with gaps that over lap the boxes with objects.
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

# Load the original image
final_annotated_image = cv2.imread(image_path)

# Define colors
object_bbox_color = (0, 255, 0)         # Green for object bounding boxes
overlapping_gap_color = (0, 0, 255)     # Red for overlapping gaps
non_overlapping_gap_color = (255, 0, 0) # Blue for non-overlapping gaps
text_color = (255, 255, 255)            # White text

font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.6
font_thickness = 1

# 1. Draw all detected object bounding boxes in green
for r in results:
    boxes = r.boxes.xyxy.cpu().numpy()
    class_ids = r.boxes.cls.cpu().numpy()
    names = r.names

    for i in range(len(boxes)):
        x1, y1, x2, y2 = map(int, boxes[i])
        class_name = names[int(class_ids[i])]

        cv2.rectangle(final_annotated_image, (x1, y1), (x2, y2), object_bbox_color, 2)
        label = f"{class_name}"
        (text_width, text_height), baseline = cv2.getTextSize(label, font, font_scale, font_thickness)
        cv2.rectangle(final_annotated_image, (x1, y1 - text_height - baseline), (x1 + text_width, y1), object_bbox_color, -1)
        cv2.putText(final_annotated_image, label, (x1, y1 - baseline), font, font_scale, text_color, font_thickness, cv2.LINE_AA)

# 2. Display 'overlapping_gaps' in red, with unique numbering
for i, gap in enumerate(overlapping_gaps):
    x1, y1, x2, y2 = map(int, gap)
    cv2.rectangle(final_annotated_image, (x1, y1), (x2, y2), overlapping_gap_color, 2)
    label = f"OL_GAP {i+1}" # Overlapping Gap
    (text_width, text_height), baseline = cv2.getTextSize(label, font, font_scale, font_thickness)
    cv2.rectangle(final_annotated_image, (x1, y1 - text_height - baseline), (x1 + text_width, y1), overlapping_gap_color, -1)
    cv2.putText(final_annotated_image, label, (x1, y1 - baseline), font, font_scale, text_color, font_thickness, cv2.LINE_AA)

# 3. Display 'non_overlapping_gaps' in blue, with unique numbering
for i, gap in enumerate(non_overlapping_gaps):
    x1, y1, x2, y2 = map(int, gap)
    cv2.rectangle(final_annotated_image, (x1, y1), (x2, y2), non_overlapping_gap_color, 2)
    label = f"NO_GAP {i+1}" # Non-Overlapping Gap
    (text_width, text_height), baseline = cv2.getTextSize(label, font, font_scale, font_thickness)
    cv2.rectangle(final_annotated_image, (x1, y1 - text_height - baseline), (x1 + text_width, y1), non_overlapping_gap_color, -1)
    cv2.putText(final_annotated_image, label, (x1, y1 - baseline), font, font_scale, text_color, font_thickness, cv2.LINE_AA)

# Display the final image
cv2_imshow(final_annotated_image)

In [ ]:
#@title Save image with identified Gaps

import os
import cv2

# Get the original image path
original_image_path = image_path

# Split the path into directory, base name, and extension
dir_name = os.path.dirname(original_image_path)
base_name = os.path.basename(original_image_path)
file_name_without_ext, ext = os.path.splitext(base_name)

# Construct the new filename with '_gaps' suffix
saved_image_name = f"{file_name_without_ext}_gaps{ext}"
saved_image_path = os.path.join(dir_name, saved_image_name)

# Save the annotated image
cv2.imwrite(saved_image_path, annotated_image_with_gaps)

print(f"Image with gap annotations saved to: {saved_image_path}")

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

# Load the original image
original_image_rows = cv2.imread(image_path)
# Make a copy to draw on
annotated_image_rows = original_image_rows.copy()

# Generate distinct colors for each row
# You can customize these colors or generate more if needed
colors = [
    (255, 0, 0),    # Blue
    (0, 255, 0),    # Green
    (0, 0, 255),    # Red
    (255, 255, 0),  # Cyan
    (255, 0, 255),  # Magenta
    (0, 255, 255),  # Yellow
    (128, 0, 0),    # Dark Blue
    (0, 128, 0),    # Dark Green
    (0, 0, 128),    # Dark Red
    (128, 128, 0),  # Dark Cyan
    (128, 0, 128),  # Dark Magenta
    (0, 128, 128),  # Dark Yellow
    (100, 200, 50), # Custom 1
    (50, 100, 200)  # Custom 2
]

# Loop through each row and draw its bounding boxes with a unique color
for i, row_indices in enumerate(rows):
    # Cycle through colors if there are more rows than predefined colors
    row_color = colors[i % len(colors)]

    # Get bounding boxes for the current row
    current_row_bboxes = [all_bboxes_list[j] for j in row_indices]

    for bbox in current_row_bboxes:
        x1, y1, x2, y2 = map(int, bbox) # Convert coordinates to integers

        # Draw rectangle for the bounding box
        cv2.rectangle(annotated_image_rows, (x1, y1), (x2, y2), row_color, 2)

        # Optional: Add a label for the row number
        label = f"Row {i+1}"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.5
        font_thickness = 1
        (text_width, text_height), baseline = cv2.getTextSize(label, font, font_scale, font_thickness)

        # Draw background rectangle for text
        cv2.rectangle(annotated_image_rows, (x1, y1 - text_height - baseline), (x1 + text_width, y1), row_color, -1)
        # Put text
        cv2.putText(annotated_image_rows, label, (x1, y1 - baseline), font, font_scale, (255, 255, 255), font_thickness, cv2.LINE_AA)

# Display the image with row annotations
cv2_imshow(annotated_image_rows)

In [ ]:
# ==============================
# 1️⃣ Install dependencies
# ==============================
!pip install ultralytics torch torchvision timm --quiet

In [ ]:


# ==============================
# 2️⃣ Import libraries
# ==============================
import torch
from ultralytics import YOLO
from PIL import Image
import torchvision.transforms as T
import timm
import os

# ==============================
# 3️⃣ Load YOLOv11 detection model
# (Replace with your trained SKU-110k weights)
# ==============================
model = YOLO("/content/best_sku110K_object_dentification_yolov11n_model.pt")  # Path to your YOLOv11 trained weights

# ==============================
# 4️⃣ Load classification model
# (Example: EfficientNet pretrained on ImageNet — replace with your product classifier)
# ==============================
classifier = timm.create_model('efficientnet_b0', pretrained=True, num_classes=1000)
classifier.eval()

# ImageNet normalization
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

# ==============================
# 5️⃣ Run detection
# ==============================
image_path = "/content/test_2660.jpg"  # Replace with your test image
results = model(image_path)

# ==============================
# 6️⃣ Crop detections and classify
# ==============================
image = Image.open(image_path).convert("RGB")
final_detections = []

for r in results:
    boxes = r.boxes.xyxy.cpu().numpy()
    confs = r.boxes.conf.cpu().numpy()

    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)
        crop = image.crop((x1, y1, x2, y2))

        # Transform for classifier
        input_tensor = transform(crop).unsqueeze(0)

        with torch.no_grad():
            preds = classifier(input_tensor)
            class_id = preds.argmax(dim=1).item()
            confidence = torch.softmax(preds, dim=1)[0, class_id].item()

        final_detections.append({
            "bbox": (x1, y1, x2, y2),
            "det_conf": float(confs[i]),
            "class_id": int(class_id),
            "class_conf": float(confidence)
        })

# ==============================
# 7️⃣ Show results
# ==============================
for det in final_detections:
    print(f"Box: {det['bbox']}, DetConf: {det['det_conf']:.2f}, "
          f"ClassID: {det['class_id']}, ClassConf: {det['class_conf']:.2f}")



In [ ]:
# final_detections_dict = {i: det for i, det in enumerate(final_detections)}

# print("Converted final_detections to dictionary format:")
# for key, value in final_detections_dict.items():
#     print(f"Detection {key}: {value}")


for r in final_de
# type(final_detections)


To manually draw bounding boxes with custom colors, you first need to extract the detection details from the `results` object. Each detection contains bounding box coordinates, the class of the detected object, and its confidence score.

In [ ]:
for r in results:
    # Get bounding box coordinates in xyxy format
    boxes = r.boxes.xyxy.cpu().numpy()
    # Get confidence scores
    confidences = r.boxes.conf.cpu().numpy()
    # Get class IDs
    class_ids = r.boxes.cls.cpu().numpy()
    # Get class names (mapping class_ids to names)
    names = r.names # This is a dict mapping class_id to class_name

    print("\n--- Detections for this image ---")
    for i in range(len(boxes)):
        x1, y1, x2, y2 = boxes[i]
        confidence = confidences[i]
        class_id = int(class_ids[i])
        class_name = names[class_id]

        print(f"Object: {class_name}, Confidence: {confidence:.2f}, Bounding Box: [{x1:.0f}, {y1:.0f}, {x2:.0f}, {y2:.0f}]")

# This cell only prints the detection data. To draw with custom colors, you would use this data with cv2.

# Trying Pinaki's Annotated Data

In [ ]:
import cv2
import pandas as pd
annotation_csv = '/content/train_0.csv'
img_path = '/content/train_0.jpg'
annotation_header = """class_name,x_center,y_center,width,height"""
# Load the image
image = cv2.imread(img_path)
print(f"Image loaded successfully from: {img_path}")
print(f"Image dimensions: {image.shape}")

# Load annotations from CSV
try:
    annotations_df = pd.read_csv(annotation_csv, sep=',', header=None)
    print(f"Annotations loaded successfully from: {annotation_csv}")
    print("First 5 rows of annotations:\n", annotations_df.head())
except FileNotFoundError:
    print(f"Error: Annotation file not found at {annotation_csv}")
except Exception as e:
    print(f"Error loading annotations: {e}")

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

# Make a copy of the image to draw on
annotated_image_from_csv = image.copy()
image_height, image_width, _ = image.shape

# Define a color for the bounding boxes (B, G, R) - e.g., Blue
bbox_color_csv = (255, 0, 0) # Blue color
text_color_csv = (255, 255, 255) # White text

# Loop through each annotation in the DataFrame
for index, row in annotations_df.iterrows():
    # Assuming the order is: class_name, x_center, y_center, width, height
    class_name = row[0]
    x_center_norm = row[1]
    y_center_norm = row[2]
    width_norm = row[3]
    height_norm = row[4]

    # Convert normalized coordinates to pixel coordinates
    # x_center, y_center, width, height (normalized to image_width, image_height)
    x_center_px = int(x_center_norm * image_width)
    y_center_px = int(y_center_norm * image_height)
    width_px = int(width_norm * image_width)
    height_px = int(height_norm * image_height)

    # Calculate top-left and bottom-right corner coordinates
    x1 = int(x_center_px - (width_px / 2))
    y1 = int(y_center_px - (height_px / 2))
    x2 = int(x_center_px + (width_px / 2))
    y2 = int(y_center_px + (height_px / 2))

    # Draw rectangle
    cv2.rectangle(annotated_image_from_csv, (x1, y1), (x2, y2), bbox_color_csv, 2)

    # Prepare label text
    label_csv = f"{class_name}"

    # Get text size for background rectangle
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.8
    font_thickness = 2
    (text_width, text_height), baseline = cv2.getTextSize(label_csv, font, font_scale, font_thickness)

    # Draw background rectangle for text
    cv2.rectangle(annotated_image_from_csv, (x1, y1 - text_height - baseline), (x1 + text_width, y1), bbox_color_csv, -1)
    # Put text (class name)
    cv2.putText(annotated_image_from_csv, label_csv, (x1, y1 - baseline), font, font_scale, text_color_csv, font_thickness, cv2.LINE_AA)

# Display the image with custom annotations from CSV
cv2_imshow(annotated_image_from_csv)

# Task
Identify and visualize empty shelf gaps that overlap with detected objects. This involves implementing an IoU (Intersection over Union) function to compare gap boxes with object bounding boxes, and then rendering the overlapping gaps in one color and non-overlapping gaps in another color on the image `/content/test_2844.jpg`.

## Implement IoU function

### Subtask:
Define a Python function to calculate the Intersection over Union (IoU) between two bounding boxes.


**Reasoning**:
The subtask requires defining a Python function to calculate the Intersection over Union (IoU) between two bounding boxes. This code block will implement the `calculate_iou` function as specified in the instructions, handling intersection, union, and edge cases like division by zero.



In [ ]:
import numpy as np

def calculate_iou(box1, box2):
    """
    Calculates the Intersection over Union (IoU) of two bounding boxes.

    Args:
        box1 (list or np.array): First bounding box in [x1, y1, x2, y2] format.
        box2 (list or np.array): Second bounding box in [x1, y1, x2, y2] format.

    Returns:
        float: The IoU value, or 0.0 if there is no overlap or union area is zero.
    """

    # Determine the coordinates of the intersection rectangle
    x1_box1, y1_box1, x2_box1, y2_box1 = box1
    x1_box2, y1_box2, x2_box2, y2_box2 = box2

    x_left = max(x1_box1, x1_box2)
    y_top = max(y1_box1, y1_box2)
    x_right = min(x2_box1, x2_box2)
    y_bottom = min(y2_box1, y2_box2)

    # Calculate the area of the intersection rectangle
    if x_right < x_left or y_bottom < y_top:
        area_intersection = 0.0
    else:
        area_intersection = (x_right - x_left) * (y_bottom - y_top)

    # Calculate the area of each bounding box
    area_box1 = (x2_box1 - x1_box1) * (y2_box1 - y1_box1)
    area_box2 = (x2_box2 - x1_box2) * (y2_box2 - y1_box2)

    # Calculate the Union area
    area_union = float(area_box1 + area_box2 - area_intersection)

    # Handle division by zero
    if area_union == 0:
        return 0.0

    # Return the IoU
    iou = area_intersection / area_union
    return iou

print("The `calculate_iou` function has been defined.")

**Reasoning**:
Now that the `calculate_iou` function is defined, the next step is to use it to categorize the detected `empty_shelf_gaps` into those that overlap with any object bounding box (`all_bboxes`) and those that do not. This involves iterating through each gap and comparing it against all object bounding boxes using a specified IoU threshold to determine overlap.



In [ ]:
overlapping_gaps = []
non_overlapping_gaps = []
iou_threshold = 0.5 # Define a threshold for considering an overlap significant

for gap_box in empty_shelf_gaps:
    is_overlapping = False
    for object_box in all_bboxes_list:
        iou = calculate_iou(gap_box, object_box)
        if iou > iou_threshold:
            is_overlapping = True
            break # Found an overlap for this gap, no need to check other objects

    if is_overlapping:
        overlapping_gaps.append(gap_box)
    else:
        non_overlapping_gaps.append(gap_box)

print(f"Total empty shelf gaps: {len(empty_shelf_gaps)}")
print(f"Gaps overlapping with objects (IoU > {iou_threshold}): {len(overlapping_gaps)}")
print(f"Gaps not overlapping with objects: {len(non_overlapping_gaps)}")

**Reasoning**:
To visualize the categorized gaps, the original image needs to be loaded. Then, the `overlapping_gaps` and `non_overlapping_gaps` will be drawn on this image with distinct colors to clearly differentiate them, as per the task requirements.



In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

# Load the original image
original_image_for_drawing_gaps = cv2.imread(image_path)
# Make a copy to draw on
annotated_image_gaps_categorized = original_image_for_drawing_gaps.copy()

# Define colors for overlapping and non-overlapping gaps
overlapping_gap_color = (0, 0, 255) # Red for overlapping gaps
non_overlapping_gap_color = (0, 255, 0) # Green for non-overlapping gaps
text_color = (255, 255, 255) # White text

font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.6
font_thickness = 1

# Draw overlapping gaps
for gap in overlapping_gaps:
    x1, y1, x2, y2 = map(int, gap)
    cv2.rectangle(annotated_image_gaps_categorized, (x1, y1), (x2, y2), overlapping_gap_color, 2)
    label = "OVERLAPPING GAP"
    (text_width, text_height), baseline = cv2.getTextSize(label, font, font_scale, font_thickness)
    cv2.rectangle(annotated_image_gaps_categorized, (x1, y1 - text_height - baseline), (x1 + text_width, y1), overlapping_gap_color, -1)
    cv2.putText(annotated_image_gaps_categorized, label, (x1, y1 - baseline), font, font_scale, text_color, font_thickness, cv2.LINE_AA)

# Draw non-overlapping gaps
for gap in non_overlapping_gaps:
    x1, y1, x2, y2 = map(int, gap)
    cv2.rectangle(annotated_image_gaps_categorized, (x1, y1), (x2, y2), non_overlapping_gap_color, 2)
    label = "EMPTY GAP"
    (text_width, text_height), baseline = cv2.getTextSize(label, font, font_scale, font_thickness)
    cv2.rectangle(annotated_image_gaps_categorized, (x1, y1 - text_height - baseline), (x1 + text_width, y1), non_overlapping_gap_color, -1)
    cv2.putText(annotated_image_gaps_categorized, label, (x1, y1 - baseline), font, font_scale, text_color, font_thickness, cv2.LINE_AA)

# Display the image with categorized gap annotations
cv2_imshow(annotated_image_gaps_categorized)


# Task
Create a comprehensive visualization on "test_2844.jpg". This visualization should include all detected object bounding boxes in green. Additionally, display 'overlapping_gaps' in red and 'non_overlapping_gaps' in blue, with each gap box uniquely numbered. Finally, ask the user to identify any gaps that are still incorrectly categorized.

## Re-categorize gaps based on IoU with objects

### Subtask:
Re-run the logic to categorize empty shelf gaps into 'overlapping' and 'non-overlapping' categories based on their Intersection over Union (IoU) with detected object bounding boxes. The IoU threshold will remain at 0.5 to signify substantial overlap. This step ensures we are using the most current categorization for visualization.


**Reasoning**:
The subtask requires re-running the logic to categorize empty shelf gaps into 'overlapping' and 'non-overlapping' categories based on their IoU with detected object bounding boxes, using an IoU threshold of 0.5. This command directly implements that logic, re-initializing the lists and performing the categorization as specified.



In [ ]:
overlapping_gaps = []
non_overlapping_gaps = []
iou_threshold = 0.5 # Define a threshold for considering an overlap significant

for gap_box in empty_shelf_gaps:
    is_overlapping = False
    for object_box in all_bboxes_list:
        iou = calculate_iou(gap_box, object_box)
        if iou > iou_threshold:
            is_overlapping = True
            break # Found an overlap for this gap, no need to check other objects

    if is_overlapping:
        overlapping_gaps.append(gap_box)
    else:
        non_overlapping_gaps.append(gap_box)

print(f"Total empty shelf gaps: {len(empty_shelf_gaps)}")
print(f"Gaps overlapping with objects (IoU > {iou_threshold}): {len(overlapping_gaps)}")
print(f"Gaps not overlapping with objects: {len(non_overlapping_gaps)}")

**Reasoning**:
The subtask of re-categorizing gaps is complete. The next logical step is to create the comprehensive visualization as per the main task, which includes drawing object bounding boxes and categorized gaps on the image with distinct colors and numbering for gaps.



In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

# Load the original image
final_annotated_image = cv2.imread(image_path)

# Define colors
object_bbox_color = (0, 255, 0)         # Green for object bounding boxes
overlapping_gap_color = (0, 0, 255)     # Red for overlapping gaps
non_overlapping_gap_color = (255, 0, 0) # Blue for non-overlapping gaps
text_color = (255, 255, 255)            # White text

font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.6
font_thickness = 1

# 1. Draw all detected object bounding boxes in green
for r in results:
    boxes = r.boxes.xyxy.cpu().numpy()
    class_ids = r.boxes.cls.cpu().numpy()
    names = r.names

    for i in range(len(boxes)):
        x1, y1, x2, y2 = map(int, boxes[i])
        class_name = names[int(class_ids[i])]

        cv2.rectangle(final_annotated_image, (x1, y1), (x2, y2), object_bbox_color, 2)
        label = f"{class_name}"
        (text_width, text_height), baseline = cv2.getTextSize(label, font, font_scale, font_thickness)
        cv2.rectangle(final_annotated_image, (x1, y1 - text_height - baseline), (x1 + text_width, y1), object_bbox_color, -1)
        cv2.putText(final_annotated_image, label, (x1, y1 - baseline), font, font_scale, text_color, font_thickness, cv2.LINE_AA)

# 2. Display 'overlapping_gaps' in red, with unique numbering
for i, gap in enumerate(overlapping_gaps):
    x1, y1, x2, y2 = map(int, gap)
    cv2.rectangle(final_annotated_image, (x1, y1), (x2, y2), overlapping_gap_color, 2)
    label = f"OL_GAP {i+1}" # Overlapping Gap
    (text_width, text_height), baseline = cv2.getTextSize(label, font, font_scale, font_thickness)
    cv2.rectangle(final_annotated_image, (x1, y1 - text_height - baseline), (x1 + text_width, y1), overlapping_gap_color, -1)
    cv2.putText(final_annotated_image, label, (x1, y1 - baseline), font, font_scale, text_color, font_thickness, cv2.LINE_AA)

# 3. Display 'non_overlapping_gaps' in blue, with unique numbering
for i, gap in enumerate(non_overlapping_gaps):
    x1, y1, x2, y2 = map(int, gap)
    cv2.rectangle(final_annotated_image, (x1, y1), (x2, y2), non_overlapping_gap_color, 2)
    label = f"NO_GAP {i+1}" # Non-Overlapping Gap
    (text_width, text_height), baseline = cv2.getTextSize(label, font, font_scale, font_thickness)
    cv2.rectangle(final_annotated_image, (x1, y1 - text_height - baseline), (x1 + text_width, y1), non_overlapping_gap_color, -1)
    cv2.putText(final_annotated_image, label, (x1, y1 - baseline), font, font_scale, text_color, font_thickness, cv2.LINE_AA)

# Display the final image
cv2_imshow(final_annotated_image)


## Final Task

### Subtask:
Provide the updated image with object detections and numbered categorized gaps. Ask the user to identify any gaps that are still incorrectly categorized.


## Summary:

### Data Analysis Key Findings
*   Out of 17 identified empty shelf gaps, **0 gaps were categorized as overlapping** with detected objects (IoU > 0.5), and consequently, **all 17 gaps were categorized as non-overlapping** with objects.
*   The visualization successfully displayed all detected object bounding boxes in green.
*   All 17 `non_overlapping_gaps` were clearly shown in blue with unique numbering, while no `overlapping_gaps` were displayed in red, consistent with the categorization results.

### Insights or Next Steps
*   The current categorization, with a 0.5 IoU threshold, indicates that none of the identified empty shelf gaps significantly overlap with existing products. This could imply efficient shelf management or that the identified gaps are truly empty spaces.
*   The next step is to present the generated annotated image to the user and prompt them to identify any gaps that appear to be incorrectly categorized, allowing for further refinement of the gap detection and classification logic.


# Task
**User's identified issue**: The user seems to have identified `NO_GAP_11` as a potentially incorrectly categorized gap.

**Plan for analyzing `NO_GAP_11`:**

1.  **Isolate `NO_GAP_11`**: Extract the bounding box coordinates for `NO_GAP_11` from the `non_overlapping_gaps` list.
2.  **Calculate IoU for `NO_GAP_11` with all objects**: Iterate through all detected object bounding boxes in `all_bboxes_list` and calculate the IoU with `NO_GAP_11`. Print these IoU values to identify any overlaps that didn't meet the `iou_threshold` of 0.5.
3.  **Prepare visualization data**: Filter `all_bboxes_list` to include only those objects that have *some* overlap (IoU > 0.0) with `NO_GAP_11`.

This will help in understanding why `NO_GAP_11` was categorized as non-overlapping and prepare for a focused visualization.

```python
# 1. Isolate NO_GAP_11 (which corresponds to index 10 in the 0-indexed list)
if len(non_overlapping_gaps) >= 11:
    no_gap_11_bbox = non_overlapping_gaps[10]
    print(f"Bounding box for NO_GAP_11: {no_gap_11_bbox}")
else:
    print("NO_GAP_11 does not exist in the non_overlapping_gaps list.")
    no_gap_11_bbox = None # Set to None to prevent errors later

# 2. Analyze IoU for NO_GAP_11 with all objects
if no_gap_11_bbox:
    print("\n--- IoU values for NO_GAP_11 with all detected objects ---")
    overlapping_objects_with_no_gap_11 = []
    for i, object_box in enumerate(all_bboxes_list):
        iou = calculate_iou(no_gap_11_bbox, object_box)
        if iou > 0.0: # Only print non-zero IoU for clarity
            print(f"  Object {i+1} BBox: {object_box}, IoU with NO_GAP_11: {iou:.4f}")
            overlapping_objects_with_no_gap_11.append(object_box)

    if not overlapping_objects_with_no_gap_11:
        print("No objects found to have any overlap (IoU > 0.0) with NO_GAP_11.")
    else:
        print(f"\nFound {len(overlapping_objects_with_no_gap_11)} objects with IoU > 0.0 with NO_GAP_11.")
        print(f"Original iou_threshold was {iou_threshold}. NO_GAP_11 was categorized as non-overlapping because no object had an IoU > {iou_threshold}.")

```

## Identify and Isolate NO_GAP_11

### Subtask:
Locate the bounding box coordinates for the specific gap identified as `NO_GAP_11` from the `non_overlapping_gaps` list. Store this bounding box for detailed analysis.


**Reasoning**:
To identify and isolate 'NO_GAP_11', I will check if the 'non_overlapping_gaps' list has enough elements and then extract the bounding box at the 10th index, storing it in 'no_gap_11_bbox'.



In [ ]:
if len(non_overlapping_gaps) >= 11:
    no_gap_11_bbox = non_overlapping_gaps[10]
    print(f"NO_GAP_11 bounding box: {no_gap_11_bbox}")
else:
    no_gap_11_bbox = None
    print("NO_GAP_11 does not exist in the non_overlapping_gaps list.")

**Reasoning**:
Now that 'NO_GAP_11' has been isolated, the next step is to calculate its IoU with all detected object bounding boxes to understand why it was categorized as non-overlapping. This will involve iterating through 'all_bboxes_list' and using the previously defined 'calculate_iou' function, printing IoU values for overlaps greater than 0.0, and storing objects that have some overlap for potential visualization.



In [ ]:
if no_gap_11_bbox: # Only proceed if NO_GAP_11 was successfully isolated
    print("\n--- IoU values for NO_GAP_11 with all detected objects ---")
    overlapping_objects_with_no_gap_11 = []
    for i, object_box in enumerate(all_bboxes_list):
        iou = calculate_iou(no_gap_11_bbox, object_box)
        if iou > 0.0: # Only print non-zero IoU for clarity and append relevant objects
            print(f"  Object {i+1} BBox: {object_box}, IoU with NO_GAP_11: {iou:.4f}")
            if iou > iou_threshold: # Check against the current iou_threshold
                overlapping_objects_with_no_gap_11.append(object_box)

    if not overlapping_objects_with_no_gap_11:
        print("No objects found to have any overlap (IoU > 0.0) with NO_GAP_11.")
    else:
        print(f"\nFound {len(overlapping_objects_with_no_gap_11)} objects with IoU > {iou_threshold} with NO_GAP_11.")
        print(f"Original iou_threshold was {iou_threshold}. NO_GAP_11 was categorized as non-overlapping because no object had an IoU > {iou_threshold}.")
else:
    print("Cannot analyze IoU for NO_GAP_11 as its bounding box was not found.")

## Visualize NO_GAP_11 and potentially overlapping objects

### Subtask:
Create a new visualization focusing on `NO_GAP_11`. Draw the original image, then `NO_GAP_11` in a distinct color, and all object bounding boxes that have *any* overlap (IoU > 0.0) with `NO_GAP_11` in another color. This will visually confirm the user's observation.


**Reasoning**:
To visually confirm the user's observation, I will create a new visualization by loading the original image and drawing 'NO_GAP_11' in a distinct color, along with all object bounding boxes that show any overlap with 'NO_GAP_11', each with appropriate labels, and then display the result.



In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

# 1. Load the original image
focused_annotated_image = cv2.imread(image_path)

# 2. Define distinct colors
no_gap_11_color = (255, 0, 255) # Magenta for NO_GAP_11
overlapping_object_color = (0, 255, 255) # Yellow for objects overlapping NO_GAP_11
text_color = (255, 255, 255) # White text

font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.6
font_thickness = 1

# 3. Draw the bounding box for NO_GAP_11
if no_gap_11_bbox:
    x1, y1, x2, y2 = map(int, no_gap_11_bbox)
    cv2.rectangle(focused_annotated_image, (x1, y1), (x2, y2), no_gap_11_color, 2)
    label = "NO_GAP_11 (User Concern)"
    (text_width, text_height), baseline = cv2.getTextSize(label, font, font_scale, font_thickness)
    cv2.rectangle(focused_annotated_image, (x1, y1 - text_height - baseline), (x1 + text_width, y1), no_gap_11_color, -1)
    cv2.putText(focused_annotated_image, label, (x1, y1 - baseline), font, font_scale, text_color, font_thickness, cv2.LINE_AA)

# 4. Iterate through overlapping_objects_with_no_gap_11 and draw each object's bounding box
if overlapping_objects_with_no_gap_11:
    for i, obj_bbox in enumerate(overlapping_objects_with_no_gap_11):
        x1, y1, x2, y2 = map(int, obj_bbox)
        cv2.rectangle(focused_annotated_image, (x1, y1), (x2, y2), overlapping_object_color, 2)
        label = f"Object (Overlap)"
        (text_width, text_height), baseline = cv2.getTextSize(label, font, font_scale, font_thickness)
        cv2.rectangle(focused_annotated_image, (x1, y1 - text_height - baseline), (x1 + text_width, y1), overlapping_object_color, -1)
        cv2.putText(focused_annotated_image, label, (x1, y1 - baseline), font, font_scale, text_color, font_thickness, cv2.LINE_AA)
else:
    print("No objects were found to have any overlap (IoU > 0.0) with NO_GAP_11.")

# 5. Display the image
cv2_imshow(focused_annotated_image)


## Propose adjustments based on analysis

### Subtask:
Based on the calculated IoU values and the visualization, suggest potential adjustments to the `iou_threshold` or modifications to the gap detection logic if necessary to correctly categorize `NO_GAP_11` and similar cases.


## Propose adjustments based on analysis

### Subtask:
Based on the calculated IoU values and the visualization, suggest potential adjustments to the `iou_threshold` or modifications to the gap detection logic if necessary to correctly categorize `NO_GAP_11` and similar cases.

### Review of `NO_GAP_11` Analysis

*   **Calculated IoU Values**: The analysis for `NO_GAP_11` showed that it had numerous (21) objects with an IoU > 0.0, with the highest individual IoU being 0.0376. None of these overlaps met the existing `iou_threshold` of 0.5.
*   **Visual Representation**: The visualization of `NO_GAP_11` clearly showed several objects occupying space within its boundaries, confirming the user's observation that it was incorrectly categorized as 'non-overlapping' despite containing objects.

### Evaluation of Current `iou_threshold` (0.5)

The `iou_threshold` of 0.5 is commonly used in object detection to signify a 'good' overlap for tasks like non-maximum suppression or true positive identification. However, for the purpose of identifying *empty* shelf gaps, a gap should be considered 'occupied' even if only a small portion of it is covered by an object. An IoU of 0.5 means that at least 50% of the combined area of the gap and the object is overlapping, which might be too strict for this specific use case. The fact that `NO_GAP_11` contained multiple objects but still registered below 0.5 IoU suggests that 0.5 is too high to capture partially occupied gaps.

### Proposed Adjustments and Rationale

**1. Adjust the `iou_threshold`:**

*   **Suggestion**: Lower the `iou_threshold` significantly. A value between `0.05` and `0.2` would likely be more appropriate. For instance, an `iou_threshold` of `0.1` (10% overlap) would likely capture `NO_GAP_11` as an 'overlapping gap' (or 'occupied gap').
*   **Rationale**: The goal is to identify truly empty spaces. If any detectable object occupies even a small portion of a supposed 'empty gap', that gap should not be categorized as completely empty. Lowering the threshold increases the sensitivity (recall) of detecting overlaps, ensuring fewer partially occupied gaps are misclassified as empty. This is crucial for inventory management or shelf planning where even small items can indicate an occupied space. The trade-off is that it might flag very minor, insignificant overlaps, potentially increasing false positives for 'occupied' gaps, but for 'empty shelf' detection, it's generally better to over-flag occupancy than to under-flag it.

**2. No immediate modification to gap detection logic:**

*   **Observation**: The `cluster_rows` and `find_gaps_in_row` functions seem to be working reasonably well in delineating potential empty regions. The issue with `NO_GAP_11` was not that it wasn't detected as a gap, but that its classification (empty vs. occupied) was incorrect due to the IoU threshold.
*   **Rationale**: Before delving into complex modifications of the gap detection itself, adjusting the IoU threshold is a more direct and impactful change to address the observed miscategorization. If, after adjusting the IoU threshold, similar problems persist where visually 'non-existent' gaps are still being created or significant 'empty' areas are being missed by the initial gap detection, then those functions would be the next area of focus.

## Final Task

### Subtask:
Present the analysis, visualization, and proposed adjustments to the user, asking for their confirmation on the identified issue and the suggested next steps.


## Summary:

### Q&A
The user's identified issue was that `NO_GAP_11` might be an incorrectly categorized gap. The analysis aimed to determine why `NO_GAP_11` was classified as non-overlapping.

`NO_GAP_11` was categorized as non-overlapping because, despite having 21 objects with some degree of overlap (IoU > 0.0), the highest calculated Intersection over Union (IoU) with any single object was approximately 0.0376. This value falls significantly below the original `iou_threshold` of 0.5, thus causing the system to classify it as non-overlapping according to the defined criteria.

### Data Analysis Key Findings
*   The bounding box for `NO_GAP_11` was identified as `[448.33176, 1504.3761, 2008.5533, 1815.2023]`.
*   A total of 21 objects were found to have some level of overlap (IoU > 0.0) with `NO_GAP_11`.
*   The highest observed IoU between `NO_GAP_11` and any single object was approximately 0.0376.
*   This highest IoU (0.0376) is significantly lower than the established `iou_threshold` of 0.5, explaining why `NO_GAP_11` was categorized as non-overlapping.
*   Visual inspection confirmed that `NO_GAP_11` indeed contained multiple objects, despite its classification as empty by the current threshold.

### Insights or Next Steps
*   The current `iou_threshold` of 0.5 is too high for accurately identifying occupied shelf gaps, as it misclassifies areas with partial object presence as empty.
*   Lower the `iou_threshold` to a range between 0.05 and 0.2 to increase the sensitivity of overlap detection, ensuring that partially occupied gaps are correctly identified.


## Define `categorize_gaps` function

### Subtask:
Define a Python function `categorize_gaps` that takes a list of gap bounding boxes, a list of object bounding boxes, and an IoU threshold. This function will return two lists: `overlapping_gaps` and `non_overlapping_gaps` based on the specified threshold.

In [ ]:
def categorize_gaps(empty_shelf_gaps, all_bboxes_list, iou_threshold):
    """
    Categorizes empty shelf gaps into overlapping and non-overlapping groups
    based on Intersection over Union (IoU) with detected object bounding boxes.

    Args:
        empty_shelf_gaps (list): A list of bounding boxes representing empty shelf gaps.
        all_bboxes_list (list): A list of bounding boxes representing detected objects.
        iou_threshold (float): The IoU threshold to determine if a gap overlaps with an object.

    Returns:
        tuple: A tuple containing two lists:
               - overlapping_gaps (list): Gaps that overlap with any object above the threshold.
               - non_overlapping_gaps (list): Gaps that do not overlap with any object above the threshold.
    """
    overlapping_gaps = []
    non_overlapping_gaps = []

    for gap_box in empty_shelf_gaps:
        is_overlapping = False
        for object_box in all_bboxes_list:
            iou = calculate_iou(gap_box, object_box)
            if iou > iou_threshold:
                is_overlapping = True
                break # Found an overlap for this gap, no need to check other objects

        if is_overlapping:
            overlapping_gaps.append(gap_box)
        else:
            non_overlapping_gaps.append(gap_box)

    return overlapping_gaps, non_overlapping_gaps

print("The `categorize_gaps` function has been defined.")

## Re-categorize gaps using the new function and updated threshold

### Subtask:
Call the `categorize_gaps` function with `empty_shelf_gaps`, `all_bboxes_list`, and a new `iou_threshold` of 0.1 to re-categorize the gaps. This ensures the categorization reflects the user's updated requirements for overlap detection.

In [ ]:
iou_threshold = 0.01 # Updated threshold as per user request
overlapping_gaps, non_overlapping_gaps = categorize_gaps(empty_shelf_gaps, all_bboxes_list, iou_threshold)

print(f"Total empty shelf gaps: {len(empty_shelf_gaps)}")
print(f"Gaps overlapping with objects (IoU > {iou_threshold}): {len(overlapping_gaps)}")
print(f"Gaps not overlapping with objects: {len(non_overlapping_gaps)}")

## Visualize all detections and categorized gaps with labels (Updated)

### Subtask:
Create a comprehensive visualization that loads the original image. It will display all detected object bounding boxes (from `all_bboxes_list`). Additionally, it will draw the `overlapping_gaps` in red and `non_overlapping_gaps` in blue, with each gap box clearly labeled with a unique number. This will allow for precise identification of specific gaps based on the new categorization.

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

# Load the original image
final_annotated_image = cv2.imread(image_path)

# Define colors
object_bbox_color = (0, 255, 0)         # Green for object bounding boxes
overlapping_gap_color = (0, 0, 255)     # Red for overlapping gaps
non_overlapping_gap_color = (255, 0, 0) # Blue for non-overlapping gaps
text_color = (255, 255, 255)            # White text

font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.6
font_thickness = 1

# 1. Draw all detected object bounding boxes in green
for r in results:
    boxes = r.boxes.xyxy.cpu().numpy()
    class_ids = r.boxes.cls.cpu().numpy()
    names = r.names

    for i in range(len(boxes)):
        x1, y1, x2, y2 = map(int, boxes[i])
        class_name = names[int(class_ids[i])]

        cv2.rectangle(final_annotated_image, (x1, y1), (x2, y2), object_bbox_color, 2)
        label = f"{class_name}"
        (text_width, text_height), baseline = cv2.getTextSize(label, font, font_scale, font_thickness)
        cv2.rectangle(final_annotated_image, (x1, y1 - text_height - baseline), (x1 + text_width, y1), object_bbox_color, -1)
        cv2.putText(final_annotated_image, label, (x1, y1 - baseline), font, font_scale, text_color, font_thickness, cv2.LINE_AA)

# 2. Display 'overlapping_gaps' in red, with unique numbering
for i, gap in enumerate(overlapping_gaps):
    x1, y1, x2, y2 = map(int, gap)
    cv2.rectangle(final_annotated_image, (x1, y1), (x2, y2), overlapping_gap_color, 2)
    label = f"OL_GAP {i+1}" # Overlapping Gap
    (text_width, text_height), baseline = cv2.getTextSize(label, font, font_scale, font_thickness)
    cv2.rectangle(final_annotated_image, (x1, y1 - text_height - baseline), (x1 + text_width, y1), overlapping_gap_color, -1)
    cv2.putText(final_annotated_image, label, (x1, y1 - baseline), font, font_scale, text_color, font_thickness, cv2.LINE_AA)

# 3. Display 'non_overlapping_gaps' in blue, with unique numbering
for i, gap in enumerate(non_overlapping_gaps):
    x1, y1, x2, y2 = map(int, gap)
    cv2.rectangle(final_annotated_image, (x1, y1), (x2, y2), non_overlapping_gap_color, 2)
    label = f"NO_GAP {i+1}" # Non-Overlapping Gap
    (text_width, text_height), baseline = cv2.getTextSize(label, font, font_scale, font_thickness)
    cv2.rectangle(final_annotated_image, (x1, y1 - text_height - baseline), (x1 + text_width, y1), non_overlapping_gap_color, -1)
    cv2.putText(final_annotated_image, label, (x1, y1 - baseline), font, font_scale, text_color, font_thickness, cv2.LINE_AA)

# Display the final image
cv2_imshow(final_annotated_image)